In [9]:
from osgeo import gdal
import geopandas as gpd
import gemgis as gg
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import json
import datetime
import os
os.chdir("D:/Projects/crime-prediction")

In [10]:
pre_processed_file_path = 'data/preprocessed/preprocessed-6.csv'
hex_grid_path = 'data/preprocessed/hex_grid.geojson'

In [11]:
def parse_preprocessed_data():
    df = pd.read_csv(
                         pre_processed_file_path,
                         parse_dates=['Offense Date'],                       
                         dtype={
                             'Precinct': 'category',
                             'Offense Category': 'category',
                             'Latitude': np.float32,
                             'Longitude': np.float32
                         }
                        )
    df["Id"] = df.index
    gdf = gpd.GeoDataFrame(df, geometry= df["geometry"].apply(wkt.loads), crs='EPSG:4326')
    return gdf

In [12]:
crime_gdf = parse_preprocessed_data()
print(len(crime_gdf))

1280004


In [13]:
estimated_utm_crs = crime_gdf.estimate_utm_crs()
crime_gdf_projected = crime_gdf.to_crs(estimated_utm_crs)

In [14]:
def parse_hex_grid():
    gdf = gpd.read_file(hex_grid_path)
    estimated_utm_crs = gdf.estimate_utm_crs()
    gdf_projected = gdf.to_crs(estimated_utm_crs)
    return gdf_projected

In [15]:
hex_grid_gdf = parse_hex_grid()

In [16]:
crime_gdf_projected = gpd.sjoin(crime_gdf_projected, hex_grid_gdf, how="inner", predicate="intersects")

In [17]:
crime_gdf['Grid Id'] = crime_gdf_projected['index_right']

In [18]:
crime_gdf.to_csv('data/preprocessed/preprocessed-8.csv', index=False)